# Imports

In [20]:
from __future__ import division, print_function, absolute_import, unicode_literals
from IPython.core.debugger import set_trace

import networkx as nx
import io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pk
from time import time
from typing import Text
import tensorflow as tf

import sys
sys.path.insert(0, '../src/')
import utils
import text_processor
import broadcast_network_extraction

In [2]:
net_extractor = broadcast_network_extraction.NetworkExtraction()
content_fixer = text_processor.FormalEnglishTranslator('../bagofwords/slang.txt')

In [3]:
directory = '/home/omid/Datasets/Jeopardy/'
data = None
try:
    data = utils.load_it(directory + 'Teams_logs.pk')
except:
    print("ERROR")

print(len(data))

49


In [5]:
combined_logs = {}
for team_id, team_log in data.items():
    print("Processing team", team_id, '...')
    this_team_nets = []
    this_team_number_of_networks = min(
        len(team_log.messages) // 5,
        len(team_log.member_influences))
    all_messages_before_appraisal_reports = []
    for i in range(this_team_number_of_networks):
        all_messages_before_appraisal_reports.append(
            pd.concat(
                [team_log.messages[i] for i in np.arange(i * 5, i * 5 + 5)]))
    if len(all_messages_before_appraisal_reports) > 0:
        combined_logs[team_id] = all_messages_before_appraisal_reports
    else:
        print('Team', team_id, 'does not have enough logs.')

Processing team 7 ...
Processing team 10 ...
Processing team 11 ...
Processing team 12 ...
Processing team 13 ...
Processing team 14 ...
Processing team 15 ...
Processing team 16 ...
Processing team 17 ...
Processing team 19 ...
Processing team 20 ...
Processing team 21 ...
Processing team 22 ...
Processing team 23 ...
Processing team 27 ...
Processing team 28 ...
Processing team 30 ...
Processing team 31 ...
Processing team 32 ...
Processing team 33 ...
Processing team 34 ...
Processing team 35 ...
Team 35 does not have enough logs.
Processing team 36 ...
Processing team 37 ...
Processing team 38 ...
Processing team 39 ...
Processing team 40 ...
Processing team 41 ...
Processing team 42 ...
Processing team 43 ...
Processing team 44 ...
Processing team 45 ...
Processing team 46 ...
Processing team 47 ...
Processing team 48 ...
Processing team 49 ...
Processing team 70 ...
Processing team 71 ...
Processing team 72 ...
Team 72 does not have enough logs.
Processing team 73 ...
Team 73 doe

In [31]:
for b in range(2, 5):
    for e in range(6, 12):
        if b >= e:
            continue
        time_window = [b, e]
        print("time window = ", time_window)
        for team_id, all_messages_before_appraisal_reports in combined_logs.items():
            members = data[team_id].members
            print("Processing team", team_id, '...')
            this_team_nets = []
            for index, all_messages_before_appraisal_report in enumerate(all_messages_before_appraisal_reports):
                reply_duration_net = net_extractor.extract_network_from_broadcast(            
                    communication_data=all_messages_before_appraisal_report,
                    time_window=time_window,
                    weight_type=broadcast_network_extraction.WeightType.REPLY_DURATION,
                    aggregation_type=broadcast_network_extraction.AggregationType.SUM,
                    gamma=0.15,
                    node_list=members)
                influence_matrix = np.matrix(data[team_id].member_influences[index])
                to_be_predicted_influence_matrix = utils.shuffle_matrix_in_given_order(
                        matrix=influence_matrix,
                        order=np.argsort(members)) / 100
                # Use reply_duration_net to predict to_be_predicted_influence_matrix
                reply_duration_net_matrix = nx.adj_matrix(reply_duration_net).todense()
                
                # DEBUG PRINTS
                print("reply_net = ", reply_duration_net_matrix)
                print("inf_mat = ", to_be_predicted_influence_matrix)
                sub = (reply_duration_net - to_be_predicted_influence_matrix)
                mse = (np.square(reply_duration_net - to_be_predicted_influence_matrix)).mean
                print(sub)
#                 set_trace()


time window =  [2, 6]
Processing team 7 ...
reply_net =  [[0.         0.         0.         0.        ]
 [1.28962986 0.         0.         0.47236655]
 [0.54881164 0.74081822 0.         0.        ]
 [0.         0.40656966 0.54881164 0.        ]]
inf_mat =  [[0.25 0.25 0.25 0.25]
 [0.01 0.97 0.01 0.01]
 [0.   0.   0.   1.  ]
 [0.4  0.2  0.4  0.  ]]
[[16.75 17.75 18.75 19.75]
 [16.99 17.03 18.99 19.99]
 [17.   18.   19.   19.  ]
 [16.6  17.8  18.6  20.  ]]
> <ipython-input-31-caa69eba09bd>(11)<module>()
      9             print("Processing team", team_id, '...')
     10             this_team_nets = []
---> 11             for index, all_messages_before_appraisal_report in enumerate(all_messages_before_appraisal_reports):
     12                 reply_duration_net = net_extractor.extract_network_from_broadcast(            
     13                     communication_data=all_messages_before_appraisal_report,

ipdb> exit


BdbQuit: 